In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import datetime
from sklearn.mixture import GaussianMixture 

****** PREPROCESSING NETWORK DATA ********

In [ ]:
#read the data
workfolder='data/';
TNet=pd.read_csv(workfolder+'taipeiD_TNet2.csv',header=-1);
TNet.head()

In [ ]:
#split into train and test
X=np.array(TNet);

In [ ]:
# check X 
print(X.shape)
X

In [ ]:
# minmax scaling of every column
for i in range(X.shape[1]):
    X[:,i]=(X[:,i]-X[:,i].min())/(X[:,i].max()-X[:,i].min())
# check x
X

In [ ]:
# Function to save
def save(filename,X):
    outfile = open(filename,'wb')
    pickle.dump(X,outfile)
    outfile.close()

In [ ]:
# Function to load
def load(filename):
    infile = open(filename,'rb')
    X=pickle.load(infile, encoding='latin1')
    infile.close()
    return X

In [ ]:
# Save taipei data
save('TaipeiExchange1.pkl',X)

In [ ]:
#XS=np.argsort(X,axis=0)

In [ ]:
#from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X=scaler.transform(X)

***** EVENTS DATA *******

In [ ]:
# Import events data
events=pd.read_csv(workfolder+'Holidays.csv', encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True);

# Check events data
print(events.shape)
print(events.info())
events.head()

In [ ]:
# Subset events
events=events.loc[events.Date<datetime.datetime(2018,10,31)]

# Check events data
print(events.shape)
print(events.info())
events.head()

In [ ]:
# Event day number from 1-jan-2017
events['Day']=(events.Date-datetime.datetime(2017,1,1)).dt.days
# Check events
events.head()

In [ ]:
# List of 'Day' for national holidays
holidays=events.Day[events['Holiday Type']=='National holiday']
# check holidays
print(len(holidays))
holidays

In [ ]:
# List of 'Day' for national holidays
otherevents=events.Day[events['Holiday Type']!='National holiday']
# check otherevents
print(len(otherevents))
otherevents

******* READ THE TRANSFORMED NETWORK DATA IN REPRESENTATION FEATURE SPACE ********

In [ ]:
#load the coded data
y=load('Tapeiexchange2.pkl')

In [ ]:
#try PCA as an alternative
from sklearn import preprocessing
from sklearn.decomposition import PCA
Xs=preprocessing.StandardScaler().fit_transform(X)
#y=preprocessing.StandardScaler().fit_transform(y)
pca = PCA(11) 
y = pca.fit_transform(Xs)
#y = pca.fit_transform(y)
#y=preprocessing.StandardScaler().fit_transform(y)

In [ ]:
y.shape

In [ ]:
#the data appers to show some clustering. Why do you think is that?
plt.scatter(y[:,3],y[:,4])

In [ ]:
#see how holidays appear in this new feature space
plt.figure()
features=[3,4]
plt.scatter(y[:,features[0]],y[:,features[1]])
plt.scatter(y[holidays,features[0]],y[holidays,features[1]])

Clearly holidays demonstrate some patter, but holiday recognition might need to be done in each cluster separately. So we'll cluster the data first

In [ ]:
pval=0.2 #sensitivity of anomaly detection; percentage of the most anomalous days to take

In [ ]:
def anomalyOutput(array, arrColumns):
    iterN=20
    rind=array[:,0]>-10 #index of regular (non-outlier pods/ints)
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))
        gm=GaussianMixture(n_components=3,n_init=100,max_iter=1000,random_state=0) #clustering model
        clustering=gm.fit(array[rind,arrColumns]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(array) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break

In [ ]:
anomalyOutput(X_pca, 0:2)

In [ ]:
print('Anomaly frequency among holidays={0:.2f}%'.format(100.0*sum(l[holidays]<=Lthres)/len(holidays)))

In [ ]:
print('Anomaly frequency among observances={0:.2f}%'.format(100.0*sum(l[otherevents]<=Lthres)/len(otherevents)))

In [ ]:
print('Anomaly frequency among other days={0:.2f}%'.format(100.0*(sum(l<Lthres)-sum(l[holidays]<=Lthres))/(len(l)-len(holidays))))